# ETL Project
## Irvine Company Apartments


In [307]:
# set environment
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [308]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\kate_\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [309]:
# build URL for scraping
complex_avail_url = "https://www.irvinecompanyapartments.com/locations/orange-county/irvine/spectrum/promenade-at-spectrum/availability.html#floor-plan-list"

In [310]:
browser.visit(complex_avail_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [311]:
parse_soup = soup
complex_soup = parse_soup.find_all("h1", class_="sticky-header__title-heading")

complex_name = complex_soup[0].text
print(complex_name)

Promenade at Irvine Spectrum


In [312]:
parse_soup = soup
floor_plans = parse_soup.find_all('div', class_="fapt-fp-list-item")



# Loop through returned results
for plan in floor_plans:
    

    # Error handling
    try:
        print(plan.find('div', class_="fapt-fp-list-item__column--plan-name").text)
        # Identify and return title of listing
        print(plan.find('div', class_="fapt-fp-list-item__column--beds-baths").text)
        # Identify and return price of listing
        print(plan.find('div', class_="fapt-fp-list-item__column--price").text)
        # Identify and return link to listing
        print(plan.find('div', class_="fapt-fp-list-item__column--sqft").text)
        
        units = plan.find_all("div", class_="fapt-fp-unit__table-row")
        
        for unit in units:
            unit_name = unit.find("span", class_="fapt-fp-unit__unit-name-text")
            if unit_name:
                print(unit_name.text)
                print(unit.find("div", class_="fapt-fp-unit__column-inner--term").text)
                print(unit.find("div", class_="fapt-fp-unit__column-inner--price").text)
                print(unit.find("div", class_="fapt-fp-unit__column-inner--available").span.text)
            


    except AttributeError as e:
        print(e)

PLAN 1
Studio / 1 Bath
 $1,935
681 
05 5314
14 mo.
$1,935
02/21/2021
04 4150
14 mo.
$1,980
02/21/2021
04 4149
14 mo.
$1,980
02/21/2021
05 5158
14 mo.
$1,980
02/21/2021
05 5114
14 mo.
$1,980
02/21/2021
PLAN 2
Studio / 1 Bath
Call for pricing 
815 
PLAN 3
1 Bed / 1 Bath
 $2,340
764 
04 4223
14 mo.
$2,340
02/21/2021
05 5350
14 mo.
$2,355
02/21/2021
05 5150
14 mo.
$2,420
02/21/2021
04 4141
14 mo.
$2,420
02/21/2021
04 4123
14 mo.
$2,420
02/21/2021
PLAN 4
1 Bed / 1 Bath
 $2,300
744 
05 5137
14 mo.
$2,300
02/21/2021
05 5133
14 mo.
$2,300
02/21/2021
05 5131
14 mo.
$2,300
02/21/2021
05 5117
14 mo.
$2,300
02/21/2021
05 5115
14 mo.
$2,300
02/21/2021
PLAN 5
1 Bed / 1 Bath
 $2,345
843 
04 4108
14 mo.
$2,345
02/21/2021
04 4339
14 mo.
$2,380
02/21/2021
04 4131
14 mo.
$2,445
02/21/2021
05 5128
14 mo.
$2,445
02/21/2021
05 5140
14 mo.
$2,445
02/21/2021
PLAN 6
1 Bed / 1 Bath
 $2,600
835 
05 5130
14 mo.
$2,600
02/21/2021
05 5230
14 mo.
$2,615
02/21/2021
05 5330
14 mo.
$2,635
02/21/2021
PLAN 7
1 Bed / 1 Ba